In [103]:
from connect_db import ConnectPostgres
from sqlalchemy import text
import pandas as pd

cp = ConnectPostgres()
engine = cp.load_engine(**cp.db_)

sql = """
select * from tteokbokki.store_review
"""
with engine.connect() as connection:
    df =  pd.read_sql(text(sql), connection)

In [104]:
df.isna().sum()

store_id                         0
user_nickname                    0
user_review_cnt                  0
user_photo_cnt                   0
user_follower_cnt                0
user_photo_path              56860
user_reservation_location    67012
user_feature                     0
user_review                  37629
user_hashtag                     0
user_visit_date                  0
user_visit_cnt                   0
user_verification                0
crawling_datetime                0
dtype: int64

- 4월 7일 이후 리뷰 제거 후, 리뷰와 비교해서 가게 정보에 있는 리뷰 수 업데이트

In [105]:
df.user_visit_date.min(), df.user_visit_date.max()

(Timestamp('2000-06-14 00:00:00'), Timestamp('2025-04-07 00:00:00'))

In [106]:
df.query("user_visit_date < @pd.Timestamp('2000-01-01')") # 데이터를 제외하고 범위가 2020년대이므로 2020, 2021, 2022년도 데이터로 간주

,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime


In [107]:
df.query("user_nickname.isin(['도로시2672', '2YE3', 'jon****']) and user_visit_date > @pd.Timestamp('2000-01-01')").groupby('user_nickname').user_visit_date.agg(['min', 'max'])

,min,max
user_nickname,,
2YE3,2021-12-20,2021-12-30
jon****,2020-09-12,2024-07-30
도로시2672,2020-09-06,2024-04-21


In [108]:
df.query("user_nickname.isin(['도로시2672', '2YE3', 'jon****'])").groupby('user_nickname').user_visit_date.agg(['min', 'max'])

,min,max
user_nickname,,
2YE3,2021-12-20,2021-12-30
jon****,2020-09-12,2024-07-30
도로시2672,2020-09-06,2024-04-21


In [109]:
df.loc[17569, 'user_visit_date'] = pd.Timestamp('2020-09-06')
df.loc[70545, 'user_visit_date'] = pd.Timestamp('2021-12-20')
df.loc[76667, 'user_visit_date'] = pd.Timestamp('2022-04-21')

In [110]:
df.user_visit_date.min(), df.user_visit_date.max()

(Timestamp('2000-06-14 00:00:00'), Timestamp('2025-04-07 00:00:00'))

In [111]:
print(df.shape)
new_df = df.query("user_visit_date < @pd.Timestamp('2025-04-08')").reset_index(drop=True) # 4월 7일까지로 
print(new_df.shape)

(79673, 14)
(79673, 14)


- 의미없는 데이터 확인하기
    - user_review랑 user_hashtag 둘 다 값이 없는 경우 => 방문 횟수를 통해서 정보를 얻을 수 있을 것으로 예상되어 보류

In [112]:
no_review = new_df.query("user_review.isna()") 
no_review_hashtag = no_review.loc[no_review.user_hashtag.map(lambda x: len(x) == 0 or (len(x) == 1 and x[0] == ''))]
print(no_review_hashtag.shape)
no_review_hashtag.head()

(14154, 14)


,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime
1,1982875847,sas****,175,3,0,None,None,[],None,[],2021-08-27,1,영수증,2025-04-07
2,1616849156,han107h,1446,61,0,None,None,[],None,[],2020-10-24,1,영수증,2025-04-07
17,35092309,김인희19,364,17,0,None,None,[],None,[],2020-05-05,1,영수증,2025-04-07
26,1982875847,hys2307,277,0,0,None,None,[],None,[],2021-07-26,1,영수증,2025-04-07
34,1982875847,oks****,291,0,0,None,None,[],None,[],2021-03-10,1,영수증,2025-04-07


- user_hashtag

In [113]:
# [혼, 밥, ///]
hashtag_wrong_split_idx = new_df.loc[new_df.user_hashtag.map(lambda x: False if len(x) == 0 else len(x[0]) == 1)].index
new_df.loc[hashtag_wrong_split_idx, 'user_hashtag'] = new_df.loc[hashtag_wrong_split_idx].user_hashtag.map(lambda x: [''.join(x)])

In [114]:
hashtag_none_idx = new_df.loc[new_df.user_hashtag.map(lambda x: len(x) == 0)].index
new_df.loc[hashtag_none_idx, 'user_hashtag'] = None

In [115]:
# ['']
hashtag_not_null = new_df.query("user_hashtag.notna()")
hashtag_wrong_space = hashtag_not_null.loc[hashtag_not_null.user_hashtag.map(lambda x: len(x) == 1 and x[0] == '')].index
new_df.loc[hashtag_wrong_space, 'user_hashtag'] =  None

- user_feature

In [124]:
feature_not_null = new_df.query("user_feature.notna()")
feature_none_idx = feature_not_null.loc[feature_not_null.user_feature.map(lambda x: len(x) == 0)].index
new_df.loc[feature_none_idx, 'user_feature'] = None

In [116]:
feature_not_null = new_df.query("user_feature.notna()")
feature_wrong_space = feature_not_null.loc[feature_not_null.user_feature.map(lambda x: len(x) == 1 and x[0] == '')].index
new_df.loc[feature_wrong_space, 'user_feature'] =  None

In [118]:
feature_not_null = new_df.query("user_feature.notna()")
new_df.loc[feature_not_null.index, "user_feature"] = feature_not_null.user_feature.map(lambda x: x.remove('') if '' in x else x)

- 펼쳐보기 부분만 제거
    - user_hashtag에서 +n 펼쳐보기가 삽입된 데이터 존재(946개)

In [120]:
import re

hashtag_not_null = new_df.query("user_hashtag.notna()")
array_two_idx = hashtag_not_null.loc[hashtag_not_null.user_hashtag.map(lambda x: len(x) == 2)].index
print(len(array_two_idx))
new_df.loc[array_two_idx, 'user_hashtag'] = new_df.loc[array_two_idx].user_hashtag.map(lambda x: x[0] if re.match(r'\+[0-9]\n개의 리뷰가 더 있습니다\n펼쳐보기', x[1]) else x)

8727


In [121]:
new_df.tail()

,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime
79668,1968521178,아이유애나23,23,38,0,None,None,"[친구, 대기 시간 바로 입장, 예약 없이 이용, 친목, 저녁에 방문]",늘 믿고 먹어요!! 파스타 맛집 ㅎㅎ,"[재료가 신선해요, 음식이 맛있어요, 양이 많아요]",2024-12-28,3,영수증,2025-04-07
79669,1658816620,3ynj,3276,465,4,None,None,[저녁에 방문],굿,[음식이 맛있어요],2022-03-27,2,영수증,2025-04-07
79670,1188501058,이주영334,1472,186,11,[./img/1188501058/이주영334_0],None,[점심에 방문],None,[재료가 신선해요],2022-10-28,1,영수증,2025-04-07
79671,1188501058,두야,629,57,3,None,None,"[대기 시간 바로 입장, 일상, 예약 없이 이용, 점심에 방문, 혼자]",가래떡 떡볶이라 좋아요,[음식이 맛있어요],2024-03-02,2,영수증,2025-04-07
79672,35092309,gon****,162,19,0,None,None,None,None,None,2020-05-12,1,영수증,2025-04-07


In [122]:
cp.save_store_review(new_df)

Pandas DataFrame이 성공적으로 저장되었습니다.
